In [ ]:
!pip uninstall -y transformers huggingface_hub accelerate
!pip install -U transformers==4.45.0 huggingface_hub==0.25.2 datasets==3.0.1 tensorflow==2.17.0

!pip install -U transformers datasets huggingface_hub tensorflow

In [ ]:
!pip install --upgrade --force-reinstall transformers==4.44.2 huggingface_hub==0.24.5 datasets tensorflow

In [ ]:
!pip install --force-reinstall "numpy<2" "matplotlib<3.9" "scikit-learn<1.5" tensorflow transformers==4.44.2 huggingface_hub==0.24.5 datasets

In [ ]:
import tensorflow as tf
from transformers import BertTokenizer, TFBertForPreTraining
from transformers import TFBertForMaskedLM, BertTokenizer
from datasets import load_dataset
import numpy as np
import random

tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
model = TFBertForMaskedLM.from_pretrained("bert-base-uncased")
dataset = load_dataset("wikitext", "wikitext-2-v1")


train_texts = [t for t in dataset["train"]["text"] if len(t.strip()) > 0]
valid_texts = [t for t in dataset["validation"]["text"] if len(t.strip()) > 0]

print(f"Loaded {len(train_texts)} training sentences and {len(valid_texts)} validation sentences.")

In [ ]:
def create_sentence_pairs(sentences, tokenizer, max_len=128):
    pairs, labels = [], []
    for i in range(len(sentences) - 1):
        # 50% positive (consecutive)
        if random.random() > 0.5:
            first, second = sentences[i], sentences[i+1]
            label = 1
        # 50% negative (random)
        else:
            first = sentences[i]
            second = random.choice(sentences)
            label = 0
        encoded = tokenizer(
            first, second,
            max_length=max_len,
            truncation=True,
            padding="max_length",
            return_tensors="tf"
        )
        pairs.append(encoded)
        labels.append(label)
    return pairs, labels

train_pairs, train_labels = create_sentence_pairs(train_texts, tokenizer)
valid_pairs, valid_labels = create_sentence_pairs(valid_texts, tokenizer)
print(f"Train pairs: {len(train_pairs)}, Validation pairs: {len(valid_pairs)}")

Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pai

In [ ]:
def mask_tokens(inputs, tokenizer, mlm_probability=0.15):
    inputs = tf.identity(inputs)
    labels = tf.identity(inputs)
    probability_matrix = tf.random.uniform(shape=inputs.shape)
    mask_arr = probability_matrix < mlm_probability

    # Don't mask special tokens
    special_tokens_mask = [
        tokenizer.get_special_tokens_mask(val, already_has_special_tokens=True)
        for val in inputs.numpy().tolist()
    ]
    mask_arr = mask_arr & ~tf.constant(special_tokens_mask, dtype=tf.bool)

    labels = tf.where(mask_arr, labels, -100)  # only compute loss on masked tokens

    # 80% MASK, 10% random, 10% unchanged
    mask_indices = tf.where(mask_arr)
    random_indices = tf.random.uniform(mask_arr.shape) < 0.1
    random_words = tf.random.uniform(inputs.shape, maxval=tokenizer.vocab_size, dtype=tf.int32)

    # Apply masking
    inputs = tf.where(mask_arr & ~random_indices, tokenizer.mask_token_id, inputs)
    inputs = tf.where(mask_arr & random_indices, random_words, inputs)
    return inputs, labels

In [ ]:
def mask_tokens(inputs, tokenizer, mlm_probability=0.15):
    """Prepare masked tokens inputs/labels for masked language modeling."""
    inputs = tf.convert_to_tensor(inputs)
    labels = tf.identity(inputs)

    # Create mask for non-special tokens
    special_tokens_mask = [
        tokenizer.get_special_tokens_mask(val, already_has_special_tokens=True)
        for val in inputs.numpy().tolist()
    ]
    # Convert to flat boolean tensor
    special_tokens_mask = np.array(special_tokens_mask).astype(bool)

    # Random mask for MLM
    probability_matrix = np.random.rand(*inputs.shape)
    mask_arr = probability_matrix < mlm_probability

    # Remove masking of special tokens
    mask_arr = mask_arr & ~special_tokens_mask

    # Masked labels
    labels = tf.where(mask_arr, labels, -100)

    # Replace masked input tokens with [MASK] token ID
    mask_token_id = tokenizer.mask_token_id
    inputs = tf.where(mask_arr, mask_token_id, inputs)

    return inputs, labels

In [ ]:
model = TFBertForPreTraining.from_pretrained("bert-base-uncased")
optimizer = tf.keras.optimizers.Adam(learning_rate=3e-5)

In [ ]:
def make_dataset(pairs, nsp_labels, batch_size=8, max_len=128):
    input_ids, attention_masks, token_type_ids, mlm_labels, nsp_labels_out = [], [], [], [], []

    for i, encoded in enumerate(pairs):
        ids, labels = mask_tokens(encoded["input_ids"], tokenizer)
        input_ids.append(ids)
        attention_masks.append(encoded["attention_mask"])
        token_type_ids.append(encoded["token_type_ids"])
        mlm_labels.append(labels)
        nsp_labels_out.append(nsp_labels[i])

    def pad(x, length=max_len):
        x = tf.convert_to_tensor(x)
        x = tf.squeeze(x)  # remove extra dimensions like [1, N]
        pad_len = length - tf.shape(x)[0]
        pad_len = tf.maximum(pad_len, 0)
        return tf.pad(x, [[0, pad_len]])[:length]  # crop if too long

    input_ids = tf.stack([pad(x) for x in input_ids])
    attention_masks = tf.stack([pad(x) for x in attention_masks])
    token_type_ids = tf.stack([pad(x) for x in token_type_ids])
    mlm_labels = tf.stack([pad(x) for x in mlm_labels])

    ds = tf.data.Dataset.from_tensor_slices((
        {
            "input_ids": input_ids,
            "attention_mask": attention_masks,
            "token_type_ids": token_type_ids,
        },
        {
            "labels": mlm_labels,
            "next_sentence_label": tf.constant(nsp_labels_out, dtype=tf.int32),
        }
    ))

    return ds.shuffle(1000).batch(batch_size)

In [7]:
train_ds = make_dataset(train_pairs[:5000], train_labels[:5000])
valid_ds = make_dataset(valid_pairs[:1000], valid_labels[:1000])

In [8]:
epochs = 2

for epoch in range(epochs):
    print(f"\n🌟 Epoch {epoch+1}/{epochs} 🌟")
    for batch, (inputs, labels) in enumerate(train_ds):
        with tf.GradientTape() as tape:
            outputs = model(**inputs, labels=labels["labels"], next_sentence_label=labels["next_sentence_label"], training=True)
            loss = outputs.loss
        grads = tape.gradient(loss, model.trainable_variables)
        optimizer.apply_gradients(zip(grads, model.trainable_variables))

        if batch % 50 == 0:
            print(f"Batch {batch}: Loss = {loss.numpy().item():.4f}")


🌟 Epoch 1/2 🌟


I0000 00:00:1762585897.770663     187 device_compiler.h:196] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


Batch 0: Loss = 7.0782
Batch 50: Loss = 2.7683
Batch 100: Loss = 2.6671
Batch 150: Loss = 1.7796
Batch 200: Loss = 1.9912
Batch 250: Loss = 2.1070
Batch 300: Loss = 2.8329
Batch 350: Loss = 2.4068
Batch 400: Loss = 1.7518
Batch 450: Loss = 2.1379
Batch 500: Loss = 2.2927
Batch 550: Loss = 2.0244
Batch 600: Loss = 2.5065

🌟 Epoch 2/2 🌟
Batch 0: Loss = 1.8407
Batch 50: Loss = 1.5433
Batch 100: Loss = 1.6225
Batch 150: Loss = 1.7414
Batch 200: Loss = 1.4320
Batch 250: Loss = 1.2757
Batch 300: Loss = 1.6896
Batch 350: Loss = 1.5605
Batch 400: Loss = 1.2133
Batch 450: Loss = 1.7342
Batch 500: Loss = 0.9593
Batch 550: Loss = 1.1996
Batch 600: Loss = 1.6255


In [10]:
val_loss_total = 0
count = 0
for inputs, labels in valid_ds:
    outputs = model(**inputs, labels=labels["labels"], next_sentence_label=labels["next_sentence_label"], training=False)
    val_loss_total += outputs.loss.numpy()
    count += 1

print(f"\n Validation Loss: {(val_loss_total / count).item():.4f}")


 Validation Loss: 2.1319


In [23]:
from transformers import BertTokenizer, TFBertForNextSentencePrediction

In [24]:
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
model = TFBertForNextSentencePrediction.from_pretrained("bert-base-uncased")

pairs = [
    ("I am coward.", "Today is beautiful day.", 1),
    ("I am coward.", "Dogs are loyal animals.", 0),
]

inputs = tokenizer(
    [a for (a, b, label) in pairs],
    [b for (a, b, label) in pairs],
    return_tensors="tf",
    truncation=True,
    padding=True,
)

outputs = model(**inputs)
logits = outputs.logits  # ✅ now this exists
probs = tf.nn.softmax(logits, axis=-1)

for i, (a, b, label) in enumerate(pairs):
    pred_label = tf.argmax(probs[i]).numpy()
    print(f"\nSentence A: {a}")
    print(f"Sentence B: {b}")
    print(f"Predicted: {'Is next' if pred_label==1 else 'Not next'} "
          f"(prob={probs[i][pred_label]:.4f}) | True: {label}")

All PyTorch model weights were used when initializing TFBertForNextSentencePrediction.

All the weights of TFBertForNextSentencePrediction were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForNextSentencePrediction for predictions without further training.



Sentence A: I am coward.
Sentence B: Today is beautiful day.
Predicted: Not next (prob=0.9998) | True: 1

Sentence A: I am coward.
Sentence B: Dogs are loyal animals.
Predicted: Not next (prob=0.9998) | True: 0
